# Machine Learning avy_wx Data (focus on deb_rail)
#### Mr. Gorton (but mostly MG)

In [ ]:
import pandas as pd

avy_wx = pd.read_pickle('data-prep/pkl/avywx2.pkl')
# avy_wx

In [ ]:
# avy_wx.info()

### Features I think we can use

Why do I think we can use these? Most of the avy obs data was observed after the target event (deb_rail) and would cause data leakage if we were to use it. 

I chose to include `ch_max_cm` because I theorize that a snow pit might be able to help determine what that value would be.

In [ ]:
fitwcu = ['avpathid', 'ch_max_cm', 'temp', 'dew_pt', 'rH', 'snow_depth', 'snowfall_3hr', 'snowfall_6hr', 'snowfall_24hr']
# fitwcu = ['avpathid', 'temp', 'dew_pt', 'rH', 'snow_depth', 'snowfall_3hr', 'snowfall_6hr', 'snowfall_24hr']
# fitwcu = ['avpathid', 'ch_max_cm']

# TODO: add wind_spd (needs to be parsed)
# TODO: date? days into season?

tts_rnd = 2 # use this to change the train_test_split random state

### Separating X & y

In [ ]:
X = avy_wx[fitwcu]
y = avy_wx['deb_rail']

### Separating Columns according to dtypes

In [ ]:
cat_cols = X.select_dtypes(include='object').columns
num_cols = X.select_dtypes(include='number').columns
other_cols = X.select_dtypes(exclude=['object', 'number']).columns

print(cat_cols)
print(num_cols)
print(other_cols)

### Splitting into X_train, y_train, X_valid, y_valid

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=tts_rnd)

### Determining Which Columns Are Missing Values

I actually don't care. The SimpleImputer will only act of NaN values and the OHE will ignore NaN's.

In [ ]:
# avy_wx[num_cols].isnull().any()

In [ ]:
# cols_missing_num = [col for col in X[num_cols] if X[col].isnull().any() == True]
# cols_missing_num

In [ ]:
# X[cat_cols].isnull().any()

In [ ]:
# cols_missing_cat = [col for col in X[cat_cols] if X[col].isnull().any() == True]
# cols_missing_cat

In [ ]:
# full_cols = [col for col in X.columns if X[col].isnull().any() == False]
# full_cols

### Imputation of Numerical Columns

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()

imputer_X_train = pd.DataFrame(imputer.fit_transform(X_train[num_cols]))
imputer_X_valid = pd.DataFrame(imputer.transform(X_valid[num_cols]))

imputer_X_train.columns = num_cols
imputer_X_valid.columns = num_cols

imputer_X_train.index = X_train.index
imputer_X_valid.index = X_valid.index

### Checking Number of Unique Values per Categorical Column

This is great for determining whether the cardinality of a column precludes one-hot encoding. In the case or `avrid`, it does not.

In [ ]:
# object_nunique = list(map(lambda col: X_train[col].nunique(), cols_missing_cat))
# d = dict(zip(cols_missing_cat, object_nunique))

# sorted(d.items(), key=lambda x: x[1])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[cat_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[cat_cols]))

OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

OH_cols_train.columns = OH_encoder.get_feature_names_out()
OH_cols_valid.columns = OH_encoder.get_feature_names_out()

# this is a little different than what MG did
full_X_train = pd.concat([imputer_X_train, OH_cols_train], axis=1)
full_X_valid = pd.concat([imputer_X_valid, OH_cols_valid], axis=1)

# use if no num cols
# full_X_train = OH_cols_train.copy()
# full_X_valid = OH_cols_valid.copy()

In [ ]:
# is this set to go?
# full_X_train.info()

### Comparing Classifiers

##### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dtc_model = DecisionTreeClassifier(random_state=0)

dtc_model.fit(full_X_train, y_train)

dtc_val_predictions = dtc_model.predict(full_X_valid)

print("Accuracy:", accuracy_score(dtc_val_predictions, y_valid))

##### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state = 1)
rf_model.fit(full_X_train, y_train)

rf_val_predictions = rf_model.predict(full_X_valid)

print("Accuracy:", accuracy_score(rf_val_predictions, y_valid))

##### Naive Bayes Classifier

##### Gradient Boosting Classifier

### Visualize Decision Tree

About the coolest thing ever

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

fig = plt.figure(figsize=(32,18))
tree.plot_tree(dtc_model, feature_names=full_X_train.columns)
plt.show()

# hi